# SPARK K-MEANS CLUSTERING

1) Instancier le client Spark Session

In [ ]:
#1 Importation
import pandas as pd
import numpy as np
import sklearn.metrics as sm
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.functions import mean, min, max
from pyspark.sql.types import IntegerType,DateType,DoubleType
import pyspark.sql.functions as F
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
import folium

# Instantiation
spark = SparkSession.builder.master("local").appName("Brisbane").getOrCreate()

2) Créer un fichier properties.conf contenant les informations relatives à vos paramètres du programme en dur

In [96]:
import configparser
config = configparser.ConfigParser()
config.read("properties.conf")

['properties.conf']

Utiliser le fichier de configuration pour récupérer les path

In [97]:
path_to_input_data = config['Bristol-City-bike']['Input-data']
path_to_output_data = config['Bristol-City-bike']['Output-data']
num_partition_kmeans = config['Bristol-City-bike']['Kmeans-level']

3) Importer le json avec spark :  bristol = spark.read.json. en utilisant la variable path-to-input-data

In [114]:
#Importation des données Json
brisbane = spark.read.json(path_to_input_data)
brisbane.show()

+--------------------+----------+----------+--------------------+------+
|             address|  latitude| longitude|                name|number|
+--------------------+----------+----------+--------------------+------+
|Lower River Tce /...|-27.482279|153.028723|122 - LOWER RIVER...|   122|
|Main St / Darragh St| -27.47059|153.036046|91 - MAIN ST / DA...|    91|
|Sydney St Ferry T...|-27.474531|153.042728|88 - SYDNEY ST FE...|    88|
|Browne St / James St|-27.461881|153.046986|75 - BROWNE ST / ...|    75|
|Kurilpa Point / M...|-27.469658|153.016696|98 - KURILPA POIN...|    98|
|Montague Rd / Ski...| -27.48172| 153.00436|109 - MONTAGUE RD...|   109|
|Macquarie St / Gu...|-27.493626|153.001482|149 - MACQUARIE S...|   149|
|Bi-centennial Bik...|-27.476076|153.002459|139 - BI-CENTENNI...|   139|
|Sir William McGre...|-27.493963|153.011938|24 - SIR WILLIAM ...|    24|
|Vulture St / Trib...|-27.482197|153.020894|117 - VULTURE ST ...|   117|
|Lamington St / Re...|-27.465226|153.050864|73 - LA

4) Création d'un nouveau data frame Kmeans-df contenant seulement les variables latitude et longitude.

In [99]:
Kmeansdf = brisbane.select("latitude","longitude")
Kmeansdf.show()

+----------+----------+
|  latitude| longitude|
+----------+----------+
|-27.482279|153.028723|
| -27.47059|153.036046|
|-27.474531|153.042728|
|-27.461881|153.046986|
|-27.469658|153.016696|
| -27.48172| 153.00436|
|-27.493626|153.001482|
|-27.476076|153.002459|
|-27.493963|153.011938|
|-27.482197|153.020894|
|-27.465226|153.050864|
|-27.468447|153.024662|
|-27.473021|153.025988|
|-27.457825|153.036866|
| -27.48148| 153.02368|
|-27.467464|153.022094|
|-27.499963|153.017633|
|-27.490776|152.994747|
|-27.458199|153.041688|
|-27.481808|153.025477|
+----------+----------+
only showing top 20 rows



In [100]:
kmeansdf = brisbane[['latitude', 'longitude']]

5) K-Means

In [101]:
num_partition_kmeans = int(num_partition_kmeans) 
features = ("longitude", "latitude")
kmeans = KMeans().setK(num_partition_kmeans).setSeed(1)
assembler = VectorAssembler(inputCols = features, outputCol = "features")
dataset = assembler.transform(Kmeansdf)
model = kmeans.fit(dataset)
fitted = model.transform(dataset)

6) Quelles sont les noms des colonnes de fitted ? Il s’agit bien de longitude, latitude, features et predictions.

In [102]:
fitted.show()

+----------+----------+--------------------+----------+
|  latitude| longitude|            features|prediction|
+----------+----------+--------------------+----------+
|-27.482279|153.028723|[153.028723,-27.4...|         2|
| -27.47059|153.036046|[153.036046,-27.4...|         2|
|-27.474531|153.042728|[153.042728,-27.4...|         1|
|-27.461881|153.046986|[153.046986,-27.4...|         1|
|-27.469658|153.016696|[153.016696,-27.4...|         2|
| -27.48172| 153.00436|[153.00436,-27.48...|         0|
|-27.493626|153.001482|[153.001482,-27.4...|         0|
|-27.476076|153.002459|[153.002459,-27.4...|         0|
|-27.493963|153.011938|[153.011938,-27.4...|         0|
|-27.482197|153.020894|[153.020894,-27.4...|         2|
|-27.465226|153.050864|[153.050864,-27.4...|         1|
|-27.468447|153.024662|[153.024662,-27.4...|         2|
|-27.473021|153.025988|[153.025988,-27.4...|         2|
|-27.457825|153.036866|[153.036866,-27.4...|         1|
| -27.48148| 153.02368|[153.02368,-27.48...|    

In [103]:
fitted

DataFrame[latitude: double, longitude: double, features: vector, prediction: int]

In [104]:
fitted.columns #vérification du noms des colonnes de fitted

['latitude', 'longitude', 'features', 'prediction']

In [105]:
# SQL
brisbane.createOrReplaceTempView("brisbaneSQL") # transformation du data frame en table 
spark.sql("""select mean(latitude) as latitude_Moyenne from brisbaneSQL""").show()
spark.sql("""select mean(longitude) as longitude_Moyenne from brisbaneSQL""").show()

#Calcul de la Latitude et Longitude moyenne 

+------------------+
|  latitude_Moyenne|
+------------------+
|-27.47130457718122|
+------------------+

+------------------+
| longitude_Moyenne|
+------------------+
|153.02508301342277|
+------------------+



7) Déterminer les longitudes et latitudes moyennes pour chaque groupe en utilisant spark DSL et SQL. Comparer les résultats

In [106]:
#Calcul de la Latitude et Longitude moyenne par groupe de prédiction
#Methode SQL

fitted.createOrReplaceTempView("fittedSQL") #transformation en table pour faire des requêtes SQL
spark.sql("""select mean(latitude) as latitude_Moyenne, prediction from fittedSQL group by prediction""")
spark.sql("""select mean(longitude) as longitude_Moyenne, prediction from fittedSQL group by prediction""")
spark.sql("""select mean(latitude) as latitude_Moyenne, mean(longitude) as longitude_Moyenne, prediction from fittedSQL group by prediction""").show()

+-------------------+------------------+----------+
|   latitude_Moyenne| longitude_Moyenne|prediction|
+-------------------+------------------+----------+
|-27.460240636363633|153.04186302272726|         1|
| -27.47255990624999|   153.02594553125|         2|
|-27.481218536585374|153.00572882926832|         0|
+-------------------+------------------+----------+



In [107]:
#Méthode DSL

fitted.groupby(col("prediction").alias("Prediction"))\
    .agg(mean(col("latitude")).alias("Moyenne_Latitude"), mean(col("longitude")).alias("Moyenne_Longitude"))\
    .show()

+----------+-------------------+------------------+
|Prediction|   Moyenne_Latitude| Moyenne_Longitude|
+----------+-------------------+------------------+
|         1|-27.460240636363633|153.04186302272726|
|         2| -27.47255990624999|   153.02594553125|
|         0|-27.481218536585374|153.00572882926832|
+----------+-------------------+------------------+



Nous trouvons bien les mêmes résultats avec les deux méthodes (DSL et SQL).

In [108]:
fitted_mean = spark.sql("""select mean(latitude) as latitude_Moyenne, mean(longitude) as longitude_Moyenne, prediction from fittedSQL group by prediction""")
fitted_mean.show()

+-------------------+------------------+----------+
|   latitude_Moyenne| longitude_Moyenne|prediction|
+-------------------+------------------+----------+
|-27.460240636363633|153.04186302272726|         1|
| -27.47255990624999|   153.02594553125|         2|
|-27.481218536585374|153.00572882926832|         0|
+-------------------+------------------+----------+



8)  Faire une visualisation dans une Map avec le package leaflet
Il s'agit d'une visualisation de la ville de Brisbane en Australie, ainsi que les 3 clusters prédits par les K-Means (la flèche orange correspond à la Prédiction 0, la bleue correspond à la Prédiction 1 et la verte à la 2)

In [109]:
#Coordonées GPS de Brisbane
Brisbane = [-27.4710107, 153.0234489] #coordonées de la ville de Brisbane

#Affiche la map
brisbane = folium.Map(location = Brisbane, zoom_start = 14.25) #affichage de la carte
folium.Marker([fitted_mean.collect()[0][0], fitted_mean.collect()[0][1]], icon=folium.Icon(color="blue"),popup = "Prédiction 1").add_to(brisbane)
folium.Marker([fitted_mean.collect()[1][0], fitted_mean.collect()[1][1]],icon=folium.Icon(color="green"), popup = "Prédiction 2").add_to(brisbane)
folium.Marker([fitted_mean.collect()[2][0], fitted_mean.collect()[2][1]],icon=folium.Icon(color="orange"), popup = "Prédiction 0").add_to(brisbane)
#ajout des flèches des clusters 0,1 et 2 à l'aide de folium Marker en récup

brisbane #on affiche la carte de Brisbane 

In [110]:
fitted.show()

+----------+----------+--------------------+----------+
|  latitude| longitude|            features|prediction|
+----------+----------+--------------------+----------+
|-27.482279|153.028723|[153.028723,-27.4...|         2|
| -27.47059|153.036046|[153.036046,-27.4...|         2|
|-27.474531|153.042728|[153.042728,-27.4...|         1|
|-27.461881|153.046986|[153.046986,-27.4...|         1|
|-27.469658|153.016696|[153.016696,-27.4...|         2|
| -27.48172| 153.00436|[153.00436,-27.48...|         0|
|-27.493626|153.001482|[153.001482,-27.4...|         0|
|-27.476076|153.002459|[153.002459,-27.4...|         0|
|-27.493963|153.011938|[153.011938,-27.4...|         0|
|-27.482197|153.020894|[153.020894,-27.4...|         2|
|-27.465226|153.050864|[153.050864,-27.4...|         1|
|-27.468447|153.024662|[153.024662,-27.4...|         2|
|-27.473021|153.025988|[153.025988,-27.4...|         2|
|-27.457825|153.036866|[153.036866,-27.4...|         1|
| -27.48148| 153.02368|[153.02368,-27.48...|    

9) Exporter la data frame fitted après élimination de la colonne  features, dans le répertoire path-to-output-data

In [111]:
fitted_final = fitted.drop('features') #on retire la colonne features et on créé un nouveau data frame fitted_final
fitted_final.show()

+----------+----------+----------+
|  latitude| longitude|prediction|
+----------+----------+----------+
|-27.482279|153.028723|         2|
| -27.47059|153.036046|         2|
|-27.474531|153.042728|         1|
|-27.461881|153.046986|         1|
|-27.469658|153.016696|         2|
| -27.48172| 153.00436|         0|
|-27.493626|153.001482|         0|
|-27.476076|153.002459|         0|
|-27.493963|153.011938|         0|
|-27.482197|153.020894|         2|
|-27.465226|153.050864|         1|
|-27.468447|153.024662|         2|
|-27.473021|153.025988|         2|
|-27.457825|153.036866|         1|
| -27.48148| 153.02368|         2|
|-27.467464|153.022094|         2|
|-27.499963|153.017633|         0|
|-27.490776|152.994747|         0|
|-27.458199|153.041688|         1|
|-27.481808|153.025477|         2|
+----------+----------+----------+
only showing top 20 rows



In [112]:
fitted_final.toPandas().to_csv(path_to_output_data + r'\fitted_final.csv') 
#on exporte fitted_final en csv vers l'emplacement cible du fichier de configuration
#Le fichier CSV est bien dans mon emplacement output
